# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [59]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [60]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [61]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [73]:
data1 = data.groupby(['CustomerID','ProductName'], as_index = False)['Quantity'].sum()
data1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [74]:
data2 = pd.pivot_table(data,values='Quantity',index=['ProductName'],columns=['CustomerID'])
data2.fillna(0,inplace=True)
data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [64]:
squareform(pdist(data2.T, 'euclidean'))

array([[  0.        ,  10.72380529,   9.64365076, ..., 197.94948851,
        218.8241303 , 203.79892051],
       [ 10.72380529,   0.        ,  10.67707825, ..., 197.34994299,
        218.62525014, 203.95342606],
       [  9.64365076,  10.67707825,   0.        , ..., 197.32460566,
        218.02981447, 203.80628057],
       ...,
       [197.94948851, 197.34994299, 197.32460566, ...,   0.        ,
        266.9269563 , 266.9269563 ],
       [218.8241303 , 218.62525014, 218.02981447, ..., 266.9269563 ,
          0.        , 269.25824036],
       [203.79892051, 203.95342606, 203.80628057, ..., 266.9269563 ,
        269.25824036,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [65]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))), 
                         index=data2.columns, columns=data2.columns)

similarities = distances[669].sort_values(ascending=False)[1:6]
similarities.head()

CustomerID
1920    0.093498
3305    0.092610
3317    0.092610
2566    0.092610
3535    0.092176
Name: 669, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [79]:
data5 = data1.loc[data1['CustomerID'].isin(similarities.index)]
data5.head()

,CustomerID,ProductName,Quantity
1681,1920,Apricots - Halves,1
1682,1920,Bacardi Breezer - Tropical,1
1683,1920,Barramundi,1
1684,1920,Beans - Kidney White,1
1685,1920,Beef - Texas Style Burger,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [67]:
data6=data5.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
data6.head()

,Quantity
ProductName,
Cod - Black Whole Fillet,5
Pork - Kidney,4
Beans - Wax,4
"Pepper - Black, Whole",4
"Cheese - Boursin, Garlic / Herbs",4


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [81]:
data7 = data2[data2[669]==0]
merged= data7.merge(data6, left_on='ProductName', right_on='ProductName').sort_values('Quantity', ascending= False)
merged33= merged[669].head()
merged33

ProductName
Cod - Black Whole Fillet    0.0
Kellogs Special K Cereal    0.0
Beans - Wax                 0.0
Pepper - Black, Whole       0.0
Tart Shells - Sweet, 4      0.0
Name: 669, dtype: float64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [86]:
d={}
lst=set(data['CustomerID'].tolist())

for i in lst:
    similarities=distances.nlargest(6, [i]) [1:6]
    data5=data1.loc[data1['CustomerID'].isin(similarities.index.tolist())]
    data6=data5.groupby(['ProductName','CustomerID'], 
                                                      as_index=False).sum().sort_values(
    'Quantity',ascending=False)
    m=data6.merge(data2, left_on='ProductName', right_on='ProductName')
    products_not_purchased=m[m[i]==0]
    x=products_not_purchased.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    
    d.update({i: x})
print(d)

{83973: ['Veal - Slab Bacon', 'Ezy Change Mophandle', 'Bandage - Fexible 1x3', 'Cocoa Butter', 'Muffin - Carrot Individual Wrap'], 49159: ['Beef - Rib Eye Aaa', 'Wine - Magnotta, Merlot Sr Vqa', 'Beer - Sleemans Cream Ale', 'Bread Fig And Almond', 'Cookies - Assorted'], 22536: ['Cheese - Mozzarella', 'Lamb - Ground', 'Rabbit - Whole', 'Soup - Campbells, Lentil', 'Phyllo Dough'], 18441: ['Bread Foccacia Whole', 'Mushroom - Trumpet, Dry', 'Island Oasis - Mango Daiquiri', 'Wiberg Super Cure', 'Sprouts - Alfalfa'], 60759: ['Bar - Granola Trail Mix Fruit Nut', 'Tomato - Tricolor Cherry', 'Cheese - Wine', 'Beef Wellington', 'Nut - Chestnuts, Whole'], 86028: ['Kellogs Special K Cereal', 'Brandy - Bar', 'Muffin - Zero Transfat', 'Juice - V8 Splash', 'Muffin - Carrot Individual Wrap'], 97282: ['Crackers - Trio', 'Cornflakes', 'Beer - Blue', 'Squid U5 - Thailand', 'Whmis - Spray Bottle Trigger'], 75791: ['Zucchini - Yellow', 'Mayonnaise - Individual Pkg', 'Wiberg Super Cure', 'Yoghurt Tubes', 'C

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [88]:
pd.DataFrame(d).T.head()

,0,1,2,3,4
33,"Mushrooms - Black, Dried",Wine - Ej Gallo Sierra Valley,Bay Leaf,Squid U5 - Thailand,Curry Paste - Madras
200,Pork - Kidney,Pomello,Wine - Charddonnay Errazuriz,Muffin - Carrot Individual Wrap,Bay Leaf
264,Pork - Kidney,Cheese - Parmesan Grated,"Salsify, Organic",Lime Cordial - Roses,Salmon Steak - Cohoe 8 Oz
356,Beans - Wax,Wine - Vidal Icewine Magnotta,Knife Plastic - White,Butter - Unsalted,Pomello
412,Juice - Lime,"Pepper - White, Ground","Pepper - Black, Whole",Soup - Campbells Tomato Ravioli,Cheese - Bocconcini


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [92]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'correlation'))), 
                         index=data2.columns, columns=data2.columns)
d={}
lst=set(data['CustomerID'].tolist())

for i in lst:
    similarities=distances2.nlargest(6, [i]) [1:6]
    data5=data1.loc[data1['CustomerID'].isin(similarities.index.tolist())]
    data6=data5.groupby(['ProductName','CustomerID'], 
                                                      as_index=False).sum().sort_values(
    'Quantity',ascending=False)
    m=data6.merge(data2, left_on='ProductName', right_on='ProductName')
    products_not_purchased=m[m[i]==0]
    x=products_not_purchased.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    
    d.update({i: x})
pd.DataFrame(d).T

,0,1,2,3,4
33,Ketchup - Tomato,"Mushrooms - Black, Dried",Vinegar - Tarragon,Cornflakes,"Pasta - Penne, Rigate, Dry"
200,Pasta - Cheese / Spinach Bauletti,Crackers - Trio,Milk Powder,Pail With Metal Handle 16l White,Pecan Raisin - Tarts
264,Chicken - Wieners,Cookie - Dough Variety,Cake - Box Window 10x10x2.5,"Bread - Roll, Soft White Round",Pastry - Cheese Baked Scones
356,"Yogurt - Blueberry, 175 Gr",Rum - Mount Gay Eclipes,Sun - Dried Tomatoes,Cod - Black Whole Fillet,Garlic - Elephant
412,Muffin Mix - Blueberry,Sprouts - Alfalfa,Apricots Fresh,Wine - Hardys Bankside Shiraz,Crackers - Trio
464,Turkey - Oven Roast Breast,"Oregano - Dry, Rubbed",Wiberg Super Cure,Lamb - Ground,Knife Plastic - White
477,Fenngreek Seed,Vinegar - Sherry,Olive - Spread Tapenade,Muffin Batt - Blueberry Passion,Lettuce - Spring Mix
639,Pail With Metal Handle 16l White,Remy Red,"Pepper - White, Ground",Lamb - Ground,"Cheese - Boursin, Garlic / Herbs"
649,Chocolate - Feathers,Cheese - Victor Et Berthold,Pail For Lid 1537,Veal - Eye Of Round,Lime Cordial - Roses
669,Bread Crumbs - Japanese Style,Cod - Black Whole Fillet,Pail For Lid 1537,Smirnoff Green Apple Twist,Lettuce - Treviso
